In [3]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import codecs
import argparse
import joblib
import pandas as pd
import pickle
from elapsedtimer import ElapsedTimer

In [8]:
global num_encoder_words
global num_decoder_words
global max_encoder_sequence_length
global max_decoder_sequence_length
global input_word_index
global target_word_index
global reverse_input_word_dict
global reverse_target_word_dict

In [12]:
global path
global num_epochs
global batch_size
global latent_dim
global num_samples
global outdir
global verbose
global mode

In [13]:
def read_input_file(path, num_samples=10e13):
    input_texts = []
    target_texts = []
    input_words = set()
    target_words = set()
    
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split('\t')
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        
        for word in input_text.split(" "):
            if word not in input_words:
                input_words.add(word)
        
        for word in target_text.split(" "):
            if word not in target_words:
                target_words.add(word)
    
    return input_texts, target_texts, input_words, target_words

In [14]:
def vocab_generation(path, num_samples, verbose=True):
    global num_encoder_words
    global num_decoder_words
    global max_encoder_sequence_length
    global max_decoder_sequence_length
    global input_word_index
    global target_word_index
    global reverse_input_word_dict
    global reverse_target_word_dict
    input_texts, target_texts, input_words, target_words = read_input_file(path, num_samples)
    input_words = sorted(list(input_words))
    target_words = sorted(list(target_words))
    num_encoder_words = len(input_words)
    num_decoder_words = len(target_words)
    max_encoder_sequence_length = max([len(txt.split(" ")) for txt in input_texts])
    max_decoder_sequence_length = max([len(txt.split(" ")) for txt in target_texts])
    if verbose == True:
        print("Number of samples: {} \n".format(len(input_texts)))
        print("Number of unique input tokens: {} \n".format(len(input_words)))
        print("Number of unique output tokens: {} \n".format(len(output_words)))
        print("Max sequence length for inputs: {} \n".format(max_encoder_sequence_length))
        print("Max sequence length for outputs: {} \n".format(max_decoder_sequence_length))
        
    input_word_index = dict([(word, i) for i, word in enumerate(input_words)])
    target_word_index = dict([(word, i) for i, word in enumerate(target_words)])
    reverse_input_word_dict = dict((i, word) for word, i in input_word_index.items())
    reverse_target_word_dict = dict((i, word) for word, i in target_word_index.items())

In [15]:
def process_input(input_texts, target_texts=None, verbose=True):
    global max_encoder_sequence_length
    global num_encoder_words
    global num_decoder_words
    global mode
    global input_word_index
    global target_word_index
    encoder_input_data = np.zeros((len(input_texts), max_encoder_sequence_length, num_encoder_words), dtype='float32')
    decoder_input_data = np.zeros((len(input_texts), max_decoder_sequence_length, num_decoder_words), dtype='float32')
    decoder_target_data = np.zeros((len(input_texts), max_decoder_sequence_length, num_decoder_words), dtype='float32')
    
    if mode == 'train':
        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, word in enumerate(input_text.split(" ")):
                try:
                    encoder_input_data[i, t, input_word_index[word]] = 1
                except:
                    print(f'word {word} encountered for the first time, skipped')
            for t, word in enumerate(target_text.split(" ")):
                decoder_input_data[i, t, target_word_index[word]] = 1
                if t > 0:
                    try:
                        decoder_target_data[i, t-1, target_word_index[word]] = 1
                    except:
                        print(f'word {word} is encountered for the first time, skipped')
        return encoder_input_data, decoder_input_data, decoder_target_data, np.array(input_texts), np.array(target_texts)
    else:
        for i, input_text in enumerate(input_texts):
            for t, word in enumerate(input_text.split(" ")):
                try:
                    encoder_input_data[i, t, input_word_index[word]] = 1
                except:
                    print(f'word {word} is encountered for the first time, skipped')
        return encoder_input_data, None, None, np.array(input_texts), None